In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip


zsh:1: command not found: wget
unzip:  cannot find or open smsspamcollection.zip, smsspamcollection.zip.zip or smsspamcollection.zip.ZIP.


In [1]:
!ls


NLP_assignment1.ipynb NLP_assignment3.ipynb
NLP_assignment2.ipynb sampletext.txt


In [3]:
import re
import math
from collections import Counter, defaultdict


In [4]:
texts = []
labels = []

with open("SMSSpamCollection", "r", encoding="utf-8") as file:
    for line in file:
        label, message = line.strip().split('\t')
        labels.append(label)
        texts.append(message.lower())

print("Total messages:", len(texts))


FileNotFoundError: [Errno 2] No such file or directory: 'SMSSpamCollection'

In [5]:
import pandas as pd

df = pd.DataFrame({"Label": labels, "Message": texts})
df.head()


,Label,Message


In [7]:
def tokenize(text):
    return re.findall(r'\b[a-z]+\b', text)


In [8]:
def train_naive_bayes(texts, labels):
    class_word_counts = defaultdict(Counter)
    class_counts = Counter(labels)
    vocabulary = set()

    for text, label in zip(texts, labels):
        words = tokenize(text)
        vocabulary.update(words)
        class_word_counts[label].update(words)

    total_docs = len(labels)
    vocab_size = len(vocabulary)

    return class_word_counts, class_counts, total_docs, vocab_size


In [27]:
class_word_counts, class_counts, total_docs, vocab_size = train_naive_bayes(texts, labels)


In [28]:
def predict(text):
    words = tokenize(text)
    scores = {}

    for label in class_counts:
        log_prob = math.log(class_counts[label] / total_docs)
        total_words = sum(class_word_counts[label].values())

        for word in words:
            word_count = class_word_counts[label][word]
            prob = (word_count + 1) / (total_words + vocab_size)
            log_prob += math.log(prob)

        scores[label] = log_prob

    return max(scores, key=scores.get), scores


In [29]:
message = "Congratulations! You have won a free prize"
predicted_class, scores = predict(message)

print("Message:", message)
print("Predicted Class:", predicted_class)
print("Scores:", scores)


Message: Congratulations! You have won a free prize
Predicted Class: spam
Scores: {'ham': -36.25624272677644, 'spam': -27.160224210295382}


In [30]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

Training set size: 4459
Testing set size: 1115


In [31]:
def predict_with_params(text, class_word_counts_train, class_counts_train, total_docs_train, vocab_size_train):
    words = tokenize(text)
    scores = {}

    for label in class_counts_train:
        log_prob = math.log(class_counts_train[label] / total_docs_train)
        total_words = sum(class_word_counts_train[label].values())

        for word in words:
            word_count = class_word_counts_train[label][word]
            prob = (word_count + 1) / (total_words + vocab_size_train)
            log_prob += math.log(prob)

        scores[label] = log_prob

    return max(scores, key=scores.get), scores

print("Defined predict_with_params function.")

Defined predict_with_params function.


In [32]:
class_word_counts_train, class_counts_train, total_docs_train, vocab_size_train = train_naive_bayes(X_train, y_train)
print("Model retrained on training data.")

Model retrained on training data.


In [33]:
y_pred = []
for text in X_test:
    predicted_class, _ = predict_with_params(text, class_word_counts_train, class_counts_train, total_docs_train, vocab_size_train)
    y_pred.append(predicted_class)

print(f"Generated predictions for {len(y_pred)} test messages.")

Generated predictions for 1115 test messages.


In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='spam')
recall = recall_score(y_test, y_pred, pos_label='spam')
f1 = f1_score(y_test, y_pred, pos_label='spam')

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.9704
Precision: 0.8625
Recall: 0.9262
F1-score: 0.8932
